# Introduction to Ray
---
(*Suggested Time To Complete: 40 minutes*)

Welcome, we're glad to have you along! This module serves as an interactive introduction to Ray, a flexible distributed computing framework built for Python with data science and machine learning practitioners in mind. Before we jump into the structure of this tutorial, let us first unpack the context of where we are coming from along with the motivation for learning Ray.

![Main Map](intro-to-ray-images/Main_Map.png)

*Figure 1*

**Context**

When we think about our daily interactions with artificial intelligence (AI), products such as recommendation systems, photo editing software, and auto-captioning on videos come to mind. In addition to these, user-facing experiences, enterprise-level use cases like reducing downtime in manufacturing, order-fulfillment optimization, and maximizing power generation in wind farms contribute to ever-growing integration of AI with the way we live. Furthermore, today's AI applications require enormous amounts of data to be trained on and machine learning models tend to grow over time. They have become so complex and infrastructure intensive that developers have no option but to distribute execution across multiple machines. However, distributing computing is hard. It requires specialized knowledge about orchestrating clusters of computers together to efficiently schedule tasks and must provide features like fault tolerance when a component fails, high availability to minimize service interruption, and autoscaling to reduce waste.

As a data scientist, machine learning pracitioner, developer, or engineer, your contribution may center on building data processing pipelines, training complicated models, running efficient hyperparameter experiments, creating simulations of agents, and/or serving your application to users. In each case, you need to choose a distributed system to support each task, but you don't want to learn a different programming language or toss out your existing toolbox. This is where Ray comes in.

**What is Ray?**

Ray is an open source, distributed execution framework that allows you to scale AI and machine learning workloads. Our goal is to keep things simple (which is enabled by a concise core API) so that you can parallelize Python programs on your laptop, cluster, cloud, or even on-premise with minimal code changes. Ray automatically handles all aspects of distributed execution including orchestration, scheduling, fault tolerance, and auto scaling so that you can scale your apps without becoming a distributed systems expert. With a rich ecosystem of distributed machine learning libraries and integrations with many important data science tools, Ray lowers the effort needed to scale compute intensive workloads.

**Notebook Outline**

We will discuss the four major **layers** that comprise Ray, namely its core engine, high-level libraries, ecosystem of integrations, and cluster deployment support (see Figure 1). In each notebook section, we will cover a layer of Ray through a basic example, and to foster active learning, you will encounter short coding excercises and discussion questions throughout the notebook to reinforce knowledge through practice. This tutorial is designed to be both narrative as well as modular, so feel free to work through in order or skip to the relevant material for you. Below, you'll find a table of contents.

**Introduction to Ray**
- Part One: Ray Core
- Part Two: Ray AIR
    - Ray Data
    - Ray Train
    - Ray Tune
    - Ray Serve
    - Ray RLlib
- Part Three: Ray Ecosystem
- Part Four: Ray Clusters
- Homework
- Next Steps

**Prerequisites**

To gain the most from this notebook, it helps if you have a working knowledge of Python as well as previous experience with machine learning. The ideal learner has minimal familiarity with Ray and is interested in leveraging Ray's simple distributed computing framework to scale AI and Python workloads.

**Learning Goals**

Upon completion of this module, you will have a intuition for:
1. a general overview and introduction to the layers of Ray
2. popular workloads best suited for each layer
3. how to navigate next steps to start scaling your own workloads

# Part One: Ray Core
---

![Ray Core](intro-to-ray-images/Ray_Core.png)

*Figure 2*

Ray Core is a low-level, distributed computing framework for Python with a concise core API, and you can think of it as the foundation that Ray's data science libraries (Ray AIR) and third-party integrations (Ray Ecosystem) are built on. This simple and general-purpose Python library enables every developer to easily build scalable, distributed systems that run on your laptop, cluster, cloud or Kubernetes.

A key strength lies in Ray Core's simple primitives: Tasks, Actors, and Objects.

**Tasks:** Ray enables you to designate functions to be executed asychronously on separate Python workers. These asynchronous Ray functions, *tasks*, can specify their resource requirements in terms of CPUs, GPUs, and custom resources which are used by the cluster scheduler to distribute tasks for parallelized execution.

**Actors:** What tasks are to functions, actors are to classes. An actor is a stateful worker and methods of the actor are scheduled on that specific worker and can access and mutate the state of that worker.

**Objects:** In Ray, tasks and actors create and compute on objects, and we refer to these objects as *remote objects* because they can be stored anywhere in a Ray cluster. We use *object references* to refer to them, and they are cached in Ray's distributed shared memory *object store*.

Ray sets up and manages clusters of computers so that you can run distributed tasks on them. For more information about how Ray clusters work in detail, jump down to Part Four of this notebook or [check out the documentation](https://docs.ray.io/en/latest/cluster/key-concepts.html#cluster-head-node). For now, the best way to understand Ray Core is to motivate it with some popular use cases and then try our hand at parallelizing a program outselves.

**Most Popular Use Cases**

- Using Ray for Highly Parallel Tasks: **description, get actual most popular use case feedback from eng**
- Parallel Model Selection: description

### 🏡 Coding Excercise: Housing Prices with `sklearn` 📈
***
Let's take a look at how we can apply Ray Core's flexible and simple API to scale a bare bones version of a common ML task: cross-validation.

Here, we have a dataset of [California Housing](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_california_housing.html) with 20,640 samples and features including `[longitude, latitude, housing_median_age, total_rooms, total_bedrooms, population, households, median_income, median_hous_value, ocean_proximity]`. Given that we want to use a linear regression model, we want to assess how the results will generalize to an unseen independent dataset, say, on new housing data coming in this year. To do this, we would try cross-validation which is a model validation technique that resamples different portions of the data to train and test a model on different iterations. After we conduct these trials, we can average the error to get an estimate of the model's predictive performance.

However, training the same model multiple times on different subsets of a dataset can take a really long time, especially if you're working with a much more complex model and larger dataset. In this example, we will first implement the sequential approach, then improve it by distributing training with Ray Core, and finally compare the code differences to highlight how minimal the changes are.

#### Import Relevant Packages

In [4]:
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics

num_trials = 1000
X, y = fetch_california_housing(return_X_y=True, as_frame=True)

#### Train 1000 Models Sequentially
Here, we will define an `sklearn` function that randomly splits our housing dataset into testing and training subsets (in the style of Monte Carlo Cross-Validation, where subsets are generated without replacement and has non-unique subsets from round to round). `sequential()` then fits a model, generates predictions, and returns the R-squared score (closer to 1 = better performance, closer to 0 = worse performance).

Then, we'll train 1000 models on these random splits, one after another, and finally print out the average of the rounds.

In [5]:
%%time

def sequential():
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    model = LinearRegression()

    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    r2 = metrics.r2_score(y_test, predictions)

    return r2

errors_seq = []

for i in range (num_trials):
    errors_seq.append(sequential())

print(sum(errors_seq) / num_trials)


0.5995951366616112
CPU times: user 17.8 s, sys: 11.5 s, total: 29.3 s
Wall time: 8.03 s


#### From Sequential -> Parallel

We just trained 1000 linear regression models in a series and averaged their R-squared values in about ~8 seconds. Let's now leverage Ray to train these models in parallel and see a runtime improvement. When we say *sequential*, this means that each process happens one after the other in a series as opposed to a *parallel* implementation where multiple operations can happen all at once.

With just a few code changes, we will modify our existing Python program to distribute it among n number of workers. Of course, this is a lightweight example, but it's illustrative of the kind of user experience you get with Ray Core's lean API.

Notice here that we need to use four API calls:

1. `ray.init()` - initialize a Ray context
2. `@ray.remote` - function or class decorator specifying that the function will be executed as a task or the class as an actor
3. `.remote()` - postfix to every remote function, remote class declaration, or invocation of a remote class method; specifies an asynchronous operation
4. `ray.get()` - return an object or list of objects from the object ID

You may notice that instead of storing the result of `train.remote()` directly into a list of `errors`, we instead store it in a list called `obj_ids`. Once you run a Ray remote function, it will immediately return an `ObjectID`. This `ObjectID` is a *promise* of future work, meaning that the actual task of the function is delegated in the background to a worker. In order to access the expected output, you need to call `ray.get()` on the `ObjectID`

![Housing Diff](intro-to-ray-images/Housing_Diff.png)

*Figure 3*

And with just a few lines of difference, we're able to parallelize quick sort without having to concern ourselves with orchestration, fault tolerance, autoscaling, or anything else that requires specialized knowledge of distributed systems.

#### Train 1000 Models in Parallel with Ray
To start, we'll import Ray (check out our [installation instructions](https://docs.ray.io/en/latest/ray-overview/installation.html)) and start a Ray cluster on our local machine that can utilize all the cores available on your computer as workers. We use `ray.is_initialized` to allow us to make sure that we only have one Ray cluster active.

In [6]:
import ray

if ray.is_initialized:
    ray.shutdown()

ray.init()

2022-10-11 14:32:45,066	INFO worker.py:1509 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8266 


Python version:,3.10.6
Ray version:,2.0.0
Dashboard:,http://127.0.0.1:8266


As illustrated above in Figure 3, we will add the decorator `@ray.remote` to our function `distributed()` to specify that it is a remote task to be run remotely. Then we call that function as `distributed.remote()` in the `for` loop to append to our list of object ids. Finally, we fetch the result outside of the loop to access the final error list (as to not *block* the launching of remote tasks asynchronously) and print out the average R-squared value.

In [7]:
%%time

@ray.remote
def distributed():
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    model = LinearRegression()

    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    r2 = metrics.r2_score(y_test, predictions)

    return r2

obj_ids = []

for i in range (num_trials):
    obj_ids.append(distributed.remote())

errors_dist = ray.get(obj_ids)

print(sum(errors_dist) / num_trials)

0.5895728400213048
CPU times: user 192 ms, sys: 75.4 ms, total: 268 ms
Wall time: 1.37 s


And now you've done it! You have distributed the training of 1000 models in a very through round of cross-validation on our California Housing dataset. Compare the runtime for each method of training 1000 models. Is this what you expected?

### 💾 Optional Excercise: Quick Sort ⚡️
***
If you would like to test your knowledge of the Ray Core concepts we covered in the housing example, try your hand at building a distributed version of the classic algorithm, quick sort.

At a glance, quick sort selects an element from the array as a pivot, partitions the array into two sub-arrays (according to whether they are less than or greater than the pivot), then sorts them recursively. You can preview an animation of quick sort in action below in Figure 4.

Quick sort is an example of a divide and conquer algorithm, a strategy of solving a large problem by recursively breaking the problem down into smaller sub-problems. By nature, this strategy lends itself well to a parallel implementation because the execution of each operation independently solves smaller instances of the same problem which is then merged into the final solution.

Let's walkthrough the mechanics a little deeper in the code sample that follows.

![Quick Sort](https://upload.wikimedia.org/wikipedia/commons/6/6a/Sorting_quicksort_anim.gif)

*Figure 4*

#### Sequential Implementation of Quick Sort
Here is the classic version that you may be familiar with below.

1. Given an array, `partition` uses the last element as the pivot and creates two subarrays of elements less than the pivot and elements greater than the pivot (`greater` will always be empty in this case).
2. A call to `quick_sort_sequential` partitions the array into two subarrays and the pivot element, then calls itself by passing in the two newly created subarrays, and finally merges the sorted answer.
3. We use the `if len(collection) <= 500000` condition because it will be useful later in the distributed version to avoid tiny tasks. We'll discuss it further in subsequent tutorials, but if you are curious, find more information [here](https://docs.ray.io/en/latest/ray-core/tips-for-first-time.html#tip-2-avoid-tiny-tasks).

Notice that each merge depends on the recursive call below it to return before completing its task.

In [ ]:
def partition(collection):
    pivot = collection.pop()
    greater, lesser = [], []
    for element in collection:
        if element > pivot:
            greater.append(element)
        else:
            lesser.append(element)
    return lesser, pivot, greater

def quick_sort_sequential(collection):
    if len(collection) <= 500000:
        return sorted(collection)
    lesser, pivot, greater = partition(collection)
    lesser = quick_sort_sequential(lesser)
    greater = quick_sort_sequential(greater)
    return lesser + [pivot] + greater

#### Your Turn: Distributed Implementation of Quick Sort

In the space below, try to apply the key concepts in the section before to create a distributed implementation of quick sort. We'll start you out with some base of code to modify, but it's up to you finish it!

In [ ]:
import ray
if ray.is_initialized:
    ray.shutdown()

### MODIFY THIS CODE BELOW ###
def quick_sort_distributed(collection):
    if len(collection) <= 500000:
        return sorted(collection)
    lesser, pivot, greater = partition(collection)
    lesser = quick_sort_sequential(lesser)
    greater = quick_sort_sequential(greater)
    return lesser + [pivot] + greater

#### Compare Both Versions on a Large Array

Let's now compare these two implementations by timing each and inspecting the results. We'll create a large random array and time both methods to see how long each takes. Note that we use `ray.put()` to store the array in the object store as to not pass the same large object as an argument repeatedly.

In [ ]:
import time
from numpy import random

size = 10_000_000

unsorted = random.randint(1000000, size=(size)).tolist()

s = time.time()
quick_sort_sequential(unsorted)
print(f"Sequential Duration: {(time.time() - s):.3f}")

s = time.time()
unsorted_obj = ray.put(unsorted)
ray.get(quick_sort_distributed.remote(unsorted_obj))
print(f"Distributed Duration: {(time.time() - s):.3f}")

### Summary
1. Introduced to Ray Core and Most Popular Workloads
2. Sequential -> Distributed Training of 1000 Models
3. Optional: Distributed Implementation of Quick Sort

#### [Key Concepts](https://docs.ray.io/en/latest/ray-core/key-concepts.html)
- Tasks
- Actors
- Objects

#### [Key API Elements in This Section](https://docs.ray.io/en/latest/ray-core/package-ref.html#python-api)
- `ray.init()`
- `@ray.remote`
- `.remote()`
- `ray.get()`
- `ray.put()`

#### Next
Now that we've covered the core engine, let's go up one layer of abstraction to look at a suite of data science libraries build on top of Ray Core to target specific machine learning workloads.

# 2. Part Two: Ray AI Runtime (AIR)
---
![Ray AIR](intro-to-ray-images/Ray_AIR.png)
*Figure 5*

Ray AI Runtime (AIR) is a unified set of libraries for distributed data processing, model training, tuning, reinforcement learning, and model serving, all in Python. Before we lay out each library and their unique jobs to be done, let's take a moment to motivate Ray AIR by taking a high-level view of the data science and machine learning workflow.

Developing a machine learning system is an iterative and often cyclical process that roughly touches on the following stages:
1. Business Needs: work with stakeholders to identify business requirements and align on which metric to optimize
2. Data Collection: source and collect data and labels
3. Feature Engineering: turn raw data into usable material
4. Model Training: the learning part of machine learning
5. Model Evaluation: try to improve upon your baseline model with hyperparameter tuning or more feature engineering or even a more relevant set of data
6. Deployment: deploy your solution to production and/or serve your model to the end user

Each of the five main native libraries that Ray AIR wraps tackles a specific piece of the ML specific tasks outlined above and because this abstraction layer is built on top of Ray Core, it is distributed by nature.

1. Ray Data: load and manipulate data
2. Ray Train: scales model training for popular ML frameworks
3. Ray Tune: scales experiment execution and hyperparameter tuning
4. Ray Serve: scales model serving
5. Ray RLlib: for distributed reinforment learning workloads

## Ray Data
Ray Datasets are the standard way to load and exchange data in Ray libraries and applications. They provide basic distributed data transformations such as map, filter, and repartition, and are compatible with a variety of file formats, data sources, and distributed frameworks.

In [ ]:
# Create a Dataset of Python objects.
ds = ray.data.range(10000)
# -> Dataset(num_blocks=200, num_rows=10000, schema=<class 'int'>)

ds.take(5)
# -> [0, 1, 2, 3, 4]

ds.count()
# -> 10000

# Create a Dataset of Arrow records.
ds = ray.data.from_items([{"col1": i, "col2": str(i)} for i in range(10000)])
# -> Dataset(num_blocks=200, num_rows=10000, schema={col1: int64, col2: string})

ds.show(5)
# -> {'col1': 0, 'col2': '0'}
# -> {'col1': 1, 'col2': '1'}
# -> {'col1': 2, 'col2': '2'}
# -> {'col1': 3, 'col2': '3'}
# -> {'col1': 4, 'col2': '4'}

ds.schema()
# -> col1: int64
# -> col2: string

### Summary
#### Key Concepts
#### Key API Elements in This Section
#### Next

## Ray Train
Ray Train is a lightweight library for distributed deep learning that allows you to easily supercharge your distributed PyTorch and TensorFlow training on Ray.

In [ ]:
import ray.train as train
from ray.train import Trainer
import torch

def train_func():
    # Setup model.
    model = torch.nn.Linear(1, 1)
    model = train.torch.prepare_model(model)
    loss_fn = torch.nn.MSELoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)

    # Setup data.
    input = torch.randn(1000, 1)
    labels = input * 2
    dataset = torch.utils.data.TensorDataset(input, labels)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=32)
    dataloader = train.torch.prepare_data_loader(dataloader)

    # Train.
    for _ in range(5):
        for X, y in dataloader:
            pred = model(X)
            loss = loss_fn(pred, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    return model.state_dict()

trainer = Trainer(backend="torch", num_workers=4)
trainer.start()
results = trainer.run(train_func)
trainer.shutdown()

print(results)

### Summary
#### Key Concepts
#### Key API Elements in This Section
#### Next

## Ray Tune
Ray Tune is a Python library for fast hyperparameter tuning at scale. Easily distribute your trial runs to quickly find the best hyperparameters.

In [ ]:
from ray import tune
 
def objective(step, alpha, beta):
   return (0.1 + alpha * step / 100)**(-1) + beta * 0.1
 
def training_function(config):
   # Hyperparameters
   alpha, beta = config["alpha"], config["beta"]
   for step in range(10):
       # Iterative training function - can be any arbitrary training procedure.
       intermediate_score = objective(step, alpha, beta)
       # Feed the score back back to Tune.
       tune.report(mean_loss=intermediate_score)
 
analysis = tune.run(
   training_function,
   config={
       "alpha": tune.grid_search([0.001, 0.01, 0.1]),
       "beta": tune.choice([1, 2, 3])
   })
 
print("Best config: ", analysis.get_best_config(
   metric="mean_loss", mode="min"))
 
# Get a dataframe for analyzing trial results.
df = analysis.results_df

### Summary
#### Key Concepts
#### Key API Elements in This Section
#### Next

## Ray Serve
Ray Serve lets you serve machine learning models in real-time or batch using a simple Python API. Serve individual models or create composite model pipelines, where you can independently deploy, update, and scale individual components.

In [ ]:
import requests

from sklearn.datasets import load_iris
from sklearn.ensemble import GradientBoostingClassifier

from ray import serve

@serve.deployment(route_prefix="/iris")
class BoostingModel:
    def __init__(self, model):
        self.model = model
        self.label_list = iris_dataset["target_names"].tolist()

    async def __call__(self, request):
        payload = await request.json()
        print(f"Received flask request with data {payload}")

        prediction = self.model.predict([payload["vector"]])[0]
        human_name = self.label_list[prediction]
        return {"result": human_name}

if __name__ == "__main__":

    # Train model.
    iris_dataset = load_iris()
    model = GradientBoostingClassifier()
    model.fit(iris_dataset["data"], iris_dataset["target"])

    # Deploy model
    serve.run(BoostingModel.bind(model))

    # Query model
    sample_request_input = {"vector": [1.2, 1.0, 1.1, 0.9]}
    response = requests.get("http://localhost:8000/iris", json=sample_request_input)
    print(response.text)
    
    # prints
    # Result:
    # {"result": "versicolor"}

### Summary
#### Key Concepts
#### Key API Elements in This Section
#### Next

## Ray RLLib
RLlib is the industry-standard reinforcement larning Python frameowrk built on Ray. Designed for quick interation and a fast path to production, it includes 25+ latest algorithms that are all implemented to run at scale and in multi-agent mode.

In [ ]:
from ray import tune
from ray.rllib.agents.ppo import PPOTrainer
 
tune.run(PPOTrainer, config={
   "env": "CartPole-v0",
   "framework": "torch",
   "log_level": "INFO"
})

### Summary
#### Key Concepts
#### Key API Elements in This Section
#### Next

# Part Three: Ray Ecosystem
---
Up until now, we've covered Ray Core, the low-level, distributed computing framework, as well as Ray AIR, the set of high-level native libraries which include Ray Data, Train, Tune, Serve, and RLlib. In addition, Ray integrates with a growing ecosystem of the most popular Python and machine learning libraries and frameworks that you may already be familiar with.

Instead of trying to create new standards, Ray allows you to scale existing workloads by unifying tools in a common interface. This interface enables you to run tasks in a distributed way, a property most of the respective backends don't have, or not to the same extent.

For example, Ray Datasets is backed by Arrow and comes with many integrations to other frameworks, such as Spark and Dask. Ray Train and RLlib are backed by the full power of Tensorflow and PyTorch. Ray Tune supports algorithms from practically every noteable HPO tool available, including Hyperopt, Optuna, Nevergrad, Ax, SigOpt, and many others. Ray Serve can be used with frameworks such as FastAPI, gradio, and Streamlit. With ample opportunity to extend current projects as well as integrate more backends in the future, a key strength of Ray is this robust design pattern for integration which you can read more about [here](https://www.anyscale.com/blog/ray-distributed-library-patterns).

For a complete list of integrations with links, go to the [Ray Ecosystem](https://docs.ray.io/en/latest/ray-overview/ray-libraries.html#) page in the docs.

### Example: HuggingFace
### Example: XGBoost
### Example: Distributed Scikit-Learn / Joblib

### Summary
#### Key Concepts
#### Key API Elements in This Section
#### Next

# Part Four: Ray Clusters
A Ray cluster consists of nodes that are connected to each other via a network. You can program against the so-called driver, the program root, which lives on the head node. The driver can run jobs, that is a collection of tasks, that are run on the nodes in the cluster. Specifically, the individual tasks of a job are run on worker processes on worker nodes.

Ray Cluster

A Ray cluster consists of a single head node and any number of connected worker nodes. The number of worker nodes may be autoscaled with application demand as specified by your Ray cluster configuration. The head node runs the autoscaler. Users can submit jobs for execution on the Ray cluster, or can interactively use the cluster by connecting to the head node and running `ray.init`.

Head Node

Every Ray cluster has one node which is designated as the head node of the cluster. The head node is identical to other worker nodes, except that it also runs singleton processes responsible for cluster management such as the autoscaler and the Ray driver processes which run Ray jobs. Ray may schedule tasks and actor on the head node just like any other worker node, unless configured otherwise.

Worker Node

Worker nodes do not run any head node management processes, and serve only to run user code in Ray tasks and actors. They participate in distributed scheduling, as well as the storage and distribution of Ray objects in cluster memory.

Autoscaling

The Ray autoscaler is a process that runs on the head node (or as a sidecar container in the head pod if using Kubernetes). When the resource demands of the Ray workload exceed the current capacity of the cluster.

### Summary
#### Key Concepts
#### Key API Elements in This Section
#### Next

# Homework
---
If you would like to practice your new skills further with some in-depth examples beyond the embedded coding excercises, take a look at this list of suggested problems:
- [Distribute a Classical Algorithm with Ray](https://github.com/ray-project/hackathon5-algo)
    - In this excercise, go to the GitHub repo linked above for details on choosing a classic algorithm implemented in Python, editing the implementation to parallelize the work with Ray, and compare your results against the sequential implementation.


# Next Steps
---
🎉 Congratulations! You have completed your first tutorial on an Introduction to Ray! We discussed the three layers of Ray: Core, AIR, and the Ecosystem. Each library in Ray AIR (Data, Train, Tune, Serve, RLLib) and the ecosystem of integrated libraries runs on Ray Core's distributed execution engine, and with Ray Clusters, you can deploy your workloads on AWS, GCP, Azure, or on Kubernetes.

From here, you can learn and get more involved with our active community of developers and researchers by checking out the following resources:
- 📖 [Ray's "Getting Started" Guides](https://docs.ray.io/en/latest/ray-overview/index.html): A collection of QuickStart Guides for every library including installation walkthrough, examples, blogs, talks, and more!
- 💻 [Official Ray Website](https://www.ray.io/): Browse the ecosystem and use this site as a hub to get the information that you need to get going and building with Ray.
- 💬 [Join the Community on Slack](https://forms.gle/9TSdDYUgxYs8SA9e8): Find friends to discuss your new learnings in our Slack space.
- 📣 [Use the Discussion Board](https://discuss.ray.io/): Ask questions, follow topics, and view announcements on this community forum.
- 🙋‍♀️ [Join a Meetup Group](https://www.meetup.com/Bay-Area-Ray-Meetup/): Tune in on meet-ups to listen to compelling talks, get to know other users, and meet the team behind Ray.
- 🪲 [Open an Issue](https://github.com/ray-project/ray/issues/new/choose): Ray is constantly evolving to improve developer experience. Submit feature requests, bug-reports, and get help via GitHub issues.